In [2]:
import pandas as pd
import numpy as np
import scipy
import os
import os.path
from matplotlib import cm
from IPython.display import display, HTML

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import calendar

In [3]:
df_regLen = pd.read_csv("Data/regLen.csv").drop(["Unnamed: 0", "index"], axis = 1)

In [17]:
#missing days interpolated
md = pd.read_csv("Data/missingdays_interpolated.csv")
md.cycle_index.unique()

array([   0.,    4.,    6.,    7.,    8.,   10.,   14.,   15.,   18.,
         20.,   23.,   25.,   26.,   30.,   31.,   32.,   36.,   38.,
         39.,   40.,   41.,   43.,   44.,   45.,   46.,   47.,   48.,
         50.,   51.,   54.,   58.,   59.,   61.,   66.,   72.,   77.,
         78.,   82.,   83.,   84.,   89.,   92.,   93.,   95.,   96.,
         97.,   98.,   99.,  100.,  102.,  103.,  104.,  106.,  109.,
        111.,  112.,  115.,  118.,  119.,  120.,  123.,  125.,  128.,
        130.,  132.,  134.,  135.,  136.,  137.,  142.,  144.,  147.,
        148.,  149.,  151.,  152.,  160.,  161.,  162.,  163.,  165.,
        166.,  167.,  169.,  170.,  171.,  172.,  173.,  174.,  176.,
        177.,  178.,  179.,  183.,  184.,  185.,  188.,  191.,  192.,
        196.,  197.,  202.,  203.,  204.,  205.,  206.,  207.,  209.,
        213.,  215.,  216.,  217.,  218.,  220.,  223.,  224.,  225.,
        226.,  227.,  228.,  229.,  232.,  237.,  239.,  241.,  242.,
        244.,  245.,

In [16]:
cc = pd.read_csv("Data/data_clean2.csv")
cc.cycle_index.unique()

array([   0.,    4.,    6.,    7.,    8.,   10.,   14.,   15.,   18.,
         20.,   23.,   25.,   26.,   30.,   31.,   32.,   36.,   38.,
         39.,   40.,   41.,   43.,   44.,   45.,   46.,   47.,   48.,
         50.,   51.,   54.,   58.,   59.,   61.,   66.,   72.,   77.,
         78.,   82.,   83.,   84.,   89.,   92.,   93.,   95.,   96.,
         97.,   98.,   99.,  100.,  102.,  103.,  104.,  106.,  109.,
        111.,  112.,  115.,  118.,  119.,  120.,  123.,  125.,  128.,
        130.,  132.,  134.,  135.,  136.,  137.,  142.,  144.,  147.,
        148.,  149.,  151.,  152.,  160.,  161.,  162.,  163.,  165.,
        166.,  167.,  169.,  170.,  171.,  172.,  173.,  174.,  176.,
        177.,  178.,  179.,  183.,  184.,  185.,  188.,  191.,  192.,
        196.,  197.,  202.,  203.,  204.,  205.,  206.,  207.,  209.,
        213.,  215.,  216.,  217.,  218.,  220.,  223.,  224.,  225.,
        226.,  227.,  228.,  229.,  232.,  237.,  239.,  241.,  242.,
        244.,  245.,

In [1]:
def plot_heatmap(timeSeries, best_method, best_metric):
    ax = sns.clustermap(timeSeries, metric=best_metric,method=best_method, col_cluster = False,figsize=(17,15))
    plt.show()

In [ ]:
timeSeries = hr29_norm.drop("cycle_day", axis = 1).T
plot_heatmap(timeSeries, "complete", "braycurtis")